In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv('train_processed.csv')

In [9]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,is_male,Embarked_Q,Embarked_S
0,0,3,22,1,0,7,1,0,1
1,1,1,38,1,0,71,0,0,0
2,1,3,26,0,0,7,0,0,1
3,1,1,35,1,0,53,0,0,1
4,0,3,35,0,0,8,1,0,1


In [10]:
X = df.drop(['Survived'], axis=1)
y = df['Survived']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train_scaled = scalar.fit_transform(X_train)
X_test_scaled = scalar.transform(X_test)

In [14]:
import xgboost as xgb
from xgboost import XGBClassifier

In [40]:
from sklearn.model_selection import RandomizedSearchCV
xg_param = {
    'n_estimators': [100, 200, 300, 400, 500,600,700],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.001, 0.005, 0.01, 0.05],
    'reg_lambda': [0.1, 0.5, 1.0]
}
xg = XGBClassifier(random_state=42)

xg_search = RandomizedSearchCV(xg, xg_param, n_iter=100, cv=5, scoring='accuracy', random_state=42)
xg_search.fit(X_train_scaled, y_train)
print("Best Random Forest Regression params:", xg_search.best_params_)


Best Random Forest Regression params: {'subsample': 0.6, 'reg_lambda': 0.1, 'reg_alpha': 0.005, 'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.8}


In [111]:
xgb_model = XGBClassifier(subsample = 0.6, reg_lambda = 1, reg_alpha = 0.22, 
                          n_estimators = 620, max_depth=7,learning_rate = 0.01,
                          gamma = 0,colsample_bytree = 1.0)
xgb_model.fit(X_train_scaled,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=620,
              n_jobs=None, num_parallel_tree=None, ...)

In [112]:
pred = xgb_model.predict(X_test_scaled)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,pred)
accuracy

0.8491620111731844

In [113]:
test_df = pd.read_csv('test.csv')

In [114]:
passenger_ids = test_df['PassengerId']

In [115]:
test_df['Age'].fillna(X_train['Age'].median(), inplace=True)

In [116]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [117]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,is_male,Embarked_Q,Embarked_S
0,0,3,22,1,0,7,1,0,1
1,1,1,38,1,0,71,0,0,0
2,1,3,26,0,0,7,0,0,1
3,1,1,35,1,0,53,0,0,1
4,0,3,35,0,0,8,1,0,1


In [119]:
test_df.drop(['PassengerId','Name','Ticket','Cabin'], axis=1,inplace=True)

In [122]:
test_df = pd.get_dummies(test_df, columns=['Sex','Embarked'], drop_first=True)

In [124]:
test_df = test_df.rename(columns={'Sex_male': ' is_male'})
test_df.head()

,Pclass,Age,SibSp,Parch,Fare,is_male,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,True,True,False
1,3,47.0,1,0,7.0000,False,False,True
2,2,62.0,0,0,9.6875,True,True,False
3,3,27.0,0,0,8.6625,True,False,True
4,3,22.0,1,1,12.2875,False,False,True


In [125]:
df = df.astype(int)
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,is_male,Embarked_Q,Embarked_S
0,0,3,22,1,0,7,1,0,1
1,1,1,38,1,0,71,0,0,0
2,1,3,26,0,0,7,0,0,1
3,1,1,35,1,0,53,0,0,1
4,0,3,35,0,0,8,1,0,1


In [126]:
test_scaled = scalar.transform(test_df)

In [128]:
predictions = xgb_model.predict(test_scaled)

In [129]:
submission_df = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': predictions
})

In [130]:
submission_df.to_csv('submission.csv', index=False)
